**Proyecto final grupo 2**

# Clases y funciones

En el siguiente apartado se recomienda definir las clases/funciones/funciones lambdas aplicables al desarrollo. Esto permitirá tener organizado la declaración de las mismas así como la actualización.

>Note: Todas las clases/funciones deben poseer la documentación correspondiente de los parámetros que recibe, argumentos, y salida de las mismas

In [ ]:
seed = 42
ruta_directorio_original = '/kaggle/input/Original'

**Librerias y pip install**

In [ ]:
pip install neptune
pip install tf-explain --quiet

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2 as cv
from tensorflow import keras
import tensorflow as tf
import matplotlib.pyplot as plt
import os
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from random import sample
import albumentations as A
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.losses import SparseCategoricalCrossentropy
from keras.optimizers import Adam
from keras.utils import plot_model
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy, SparseCategoricalAccuracy
import neptune
from neptune.integrations.tensorflow_keras import NeptuneCallback
from neptune.types import File
from tf_explain.core import GradCAM
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
from keras.applications.resnet import ResNet50

**Neptune**

In [ ]:
#Init Neptune
run = neptune.init_run(
        project="joacodominguez/proyecto-final",
        api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI3M2Y4OGJkNC00ZjlmLTQ5MmUtYTg5YS1mMGEzMjEzZmE3Y2QifQ==", # your credentials
)

In [ ]:
model_version = neptune.init_model_version(
    model="PROY-PROY",
    project="joacodominguez/proyecto-final",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI3M2Y4OGJkNC00ZjlmLTQ5MmUtYTg5YS1mMGEzMjEzZmE3Y2QifQ==", # your credentials
)

## Carga de datos

In [ ]:
os.chdir('/kaggle/input/Original')
print(os.listdir())

def count_files(folder_names = []):
    for f_name in folder_names:
        count = len(os.listdir(f'{f_name}'))
        print(f'Folder: {f_name} contains {count} images')
        
count_files(['Benign', 'Early', 'Pre', 'Pro'])subcarpeta = ['Benign', 'Early', 'Pre', 'Pro']  # Asumo que esta es tu lista de subcarpetas

unique_dimensions = set()

for sub in subcarpeta:
    folder_path = os.path.join(ruta_directorio_original, sub)

    for file in os.listdir(folder_path):
        image_path = os.path.join(folder_path, file)
        with Image.open(image_path) as img:
            unique_dimensions.add(img.size)

if len(unique_dimensions) == 1:
    print(f"\nTodas las imágenes tienen las mismas dimensiones: {unique_dimensions.pop()}")
else:
    print(f"\nSe encontraron {len(unique_dimensions)} dimensiones únicas de imágenes: {unique_dimensions}")

In [ ]:
# Inicializa una lista para almacenar nombres de archivos de todas las subcarpetas
nombres_archivos = []

# Itera sobre las subcarpetas y obtén los nombres de archivos
for subcarpeta in os.listdir(ruta_directorio_original):
    ruta_subcarpeta = os.path.join(ruta_directorio_original, subcarpeta)
    if os.path.isdir(ruta_subcarpeta):
        nombres_archivos.extend(os.listdir(ruta_subcarpeta))

# Asegúrate de que haya al menos 30 imágenes disponibles
if len(nombres_archivos) >= 30:
    # Selecciona al azar 30 imágenes de la lista
    imagenes_ejemplo = sample(nombres_archivos, 30)

# Configura el diseño de la figura
    filas, columnas = 5, 6
    figura, ejes = plt.subplots(filas, columnas, figsize=(15, 12))
    figura.subplots_adjust(hspace=0.5)

    # Itera sobre las imágenes de ejemplo y muéstralas en la cuadrícula
    for i, nombre_archivo in enumerate(imagenes_ejemplo):
        # Itera sobre las subcarpetas para encontrar la imagen
        for subcarpeta in os.listdir(ruta_directorio_original):
            ruta_subcarpeta = os.path.join(ruta_directorio_original, subcarpeta)
            ruta_imagen = os.path.join(ruta_subcarpeta, nombre_archivo)
            
            if os.path.isfile(ruta_imagen):
                imagen = cv.imread(ruta_imagen)
                imagen = cv.cvtColor(imagen, cv.COLOR_BGR2RGB)  # Convertir de BGR a RGB para mostrar con matplotlib
                ejes[i // columnas, i % columnas].imshow(imagen)
                ejes[i // columnas, i % columnas].axis('off')
                ejes[i // columnas, i % columnas].set_title(subcarpeta)
                break  # Rompe el bucle una vez que se encuentra la imagen

    plt.show()
else:
    print("No hay suficientes imágenes en las subcarpetas para mostrar 30 ejemplos.")

## Normalizacion y preprocesamiento

In [ ]:
def preprocessor(image):
    # Applying normalization to the image
    image = cv.normalize(image, None, 0, 255, cv.NORM_MINMAX)    
    
    image = np.array(image)/255.0
    return image

## MODELADO, Train/Test/Split

In [ ]:
all_data = []

# Crear un diccionario con el nombre de la subcarpeta y su índice
label_mapping = {subcarpeta: i for i, subcarpeta in enumerate(subcarpetas)}

# Itera sobre cada subcarpeta
for subcarpeta in subcarpetas:
    ruta_subcarpeta = os.path.join(ruta_directorio_original, subcarpeta)

    # Asegúrate de que sea una subcarpeta y no un archivo
    if os.path.isdir(ruta_subcarpeta):
        # Itera sobre cada imagen en la subcarpeta
        for nombre_archivo in os.listdir(ruta_subcarpeta):
            ruta_imagen = os.path.join(ruta_subcarpeta, nombre_archivo)

            # Asegúrate de que sea un archivo y no una subcarpeta
            if os.path.isfile(ruta_imagen):
                # Lee la imagen
                imagen = cv.imread(ruta_imagen)
                imagen = cv.resize(imagen, (224, 224)) / 255.0
                
                # Mapeo de la etiqueta a valor numérico usando el diccionario label_mapping
                etiqueta = label_mapping[subcarpeta]

                all_data.append([imagen, etiqueta])

In [ ]:
# Convertir la lista a arrays de Numpy
x = np.array(img_matrix_list)
y = np.array([label_mapping[etiqueta] for etiqueta in etiquetas_list])

# Realizar la división entre conjuntos de entrenamiento, prueba y validación
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=seed)

# Dividir adicionalmente el conjunto de entrenamiento en un conjunto de validación
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=seed)

print(x_train.shape, x_test.shape, x_val.shape, y_train.shape, y_test.shape, y_val.shape)

## Data augmentation (albumentation)

In [ ]:
def plot_multiple_img(img_matrix_list, title_list, ncols, main_title=""):
    fig, axis = plt.subplots(figsize=(15, 8), nrows=2, ncols=ncols, squeeze=False)
    fig.suptitle(main_title, fontsize=18)
    fig.subplots_adjust(wspace=0.3)
    fig.subplots_adjust(hspace=0.3)

    for i, (img, title) in enumerate(zip(img_matrix_list, title_list)):
        axis[i // ncols][i % ncols].imshow(img)
        axis[i // ncols][i % ncols].set_title(title, fontsize=15)

    plt.show()

In [ ]:
# Definir la lista de transformaciones de Albumentations
albumentation_list = [
    A.RandomFog(p=1),
    A.VerticalFlip(p=1),
    A.RandomBrightness(p=1),
    A.RandomContrast(limit=0.6, p=1),
    A.RandomCrop(p=1, height=224, width=224),
    A.Rotate(p=1, limit=90),
    A.RGBShift(p=1)
]

# Obtener la lista de nombres de subcarpetas (clases)
subcarpetas = os.listdir(ruta_directorio_original)

# Lista para almacenar las imágenes originales y augmentadas
img_matrix_list = []
etiquetas_list =[]

# Iterar sobre cada subcarpeta
for subcarpeta in subcarpetas:
    ruta_subcarpeta = os.path.join(ruta_directorio_original, subcarpeta)

    # Asegurarse de que sea una subcarpeta y no un archivo
    if os.path.isdir(ruta_subcarpeta):
        # Iterar sobre cada imagen en la subcarpeta
        for nombre_archivo in os.listdir(ruta_subcarpeta):
            ruta_imagen = os.path.join(ruta_subcarpeta, nombre_archivo)

            # Asegurarse de que sea un archivo y no una subcarpeta
            if os.path.isfile(ruta_imagen):
                # Leer la imagen original
                chosen_image = cv.imread(ruta_imagen)

                # Aplicar augmentations y almacenar las imágenes
                for aug_type in albumentation_list:
                    augmented_img = aug_type(image=chosen_image)['image']
                    img_matrix_list.append(augmented_img)
                    etiquetas_list.append(subcarpeta)

                img_matrix_list.append(chosen_image)  # Agregar la imagen original
                etiquetas_list.append(subcarpeta)
# Títulos para las imágenes
titles_list = ["Original", "RandomFog", "VerticalFlip", "RandomContrast", "RandomBrightness", "RandomCrop", "Rotate", "RGBShift"]

# Mostrar las imágenes en una cuadrícula
plot_multiple_img(img_matrix_list, titles_list, ncols=4, main_title="Different Types of Augmentations")

## Desarrollo de la Arquitectura CNN

In [ ]:
tf.keras.utils.set_random_seed(seed)

In [ ]:
# Define el modelo
modelo = Sequential()
modelo.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3),padding = 'same'))
modelo.add(MaxPooling2D((2, 2)))
modelo.add(Conv2D(64, (3, 3), activation='relu',padding='same'))
modelo.add(MaxPooling2D((2, 2)))
modelo.add(Conv2D(128, (3, 3), activation='relu',padding='same'))
modelo.add(MaxPooling2D((2, 2)))
modelo.add(Flatten())
modelo.add(Dense(128, activation='relu'))
modelo.add(Dropout(0.5))  # Agrega Dropout con una tasa del 50% (ajusta según sea necesario)
modelo.add(Dense(4, activation='softmax')) 

In [ ]:
modelo.compile(optimizer='Adam',
              loss=SparseCategoricalCrossentropy(),
              metrics=['sparse_categorical_accuracy'])

In [ ]:
modelo.summary()

## Definiendo Callbacks

In [ ]:
# Neptune parameters
parameters = {"dense_units": 4,
              "activation": "relu",
              "batch_size": 64,
              "n_epochs": 20
              }

run["model/parameters"] = parameters

In [ ]:
callbacks = [
    EarlyStopping(monitor='recall', min_delta=0, patience=8, mode='auto'),
    ModelCheckpoint('best_model2.h5', monitor='val_recall', save_best_only=True),
    NeptuneCallback(run=run)
]

In [ ]:
plot_model(modelo, to_file='/kaggle/working/cnn.png')

In [ ]:
history = modelo.fit(x_train, y_train,
                    epochs=20,
                    batch_size=64,
                    validation_data=(x_val, y_val),
                    callbacks=callbacks)  # Sin envolver en otra lista

## Evaluación del modelo

In [ ]:
fig = plt.figure()
plt.plot(history.history['loss'], color='teal', label='loss')
plt.plot(history.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(history.history['sparse_categorical_accuracy'], color='teal', label='acc')
plt.plot(history.history['val_sparse_categorical_accuracy'], color='orange', label='val_acc')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
modelo.save('/kaggle/working/cnn_model.h5')
#model_resnet.save('/kaggle/working/resnet_model.h5')

In [ ]:
test_loss, test_accuracy = modelo.evaluate(x_test,y_test)
print(f'Test accuracy: {test_accuracy:.2f}, Test loss: {test_loss:.2f}')

In [ ]:
"""run["evaluation/accuracy"] = test_accuracy
run["evaluation/loss"] = test_loss
model_version["model"].upload("best_model.h5") """

In [ ]:
# Obtener predicciones en el conjunto de prueba
y_pred = modelo.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Obtener la matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred_classes)

# Visualizar la matriz de confusión
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=subcarpetas, yticklabels=subcarpetas)
plt.title('Matriz de Confusión')
plt.xlabel('Predicciones')
plt.ylabel('Etiquetas Verdaderas')
plt.show()

# Imprimir el reporte de clasificación
print(classification_report(y_test, y_pred_classes, target_names=subcarpetas))

In [ ]:
# Calcular métricas
specificity = conf_matrix[0, 0] / (conf_matrix[0, 0] + conf_matrix[0, 1])
false_positive_rate = conf_matrix[0, 1] / (conf_matrix[0, 0] + conf_matrix[0, 1])

# Visualización
plt.figure(figsize=(8, 6))
sns.barplot(x=['Especificidad', 'Tasa de Falsos Positivos'], y=[specificity, false_positive_rate], palette="viridis")
plt.title('Especificidad y Tasa de Falsos Positivos')
plt.ylabel('Valor')
plt.show()

In [ ]:
# Obtén las probabilidades predichas para cada clase
y_prob = modelo.predict(x_test)

# Binariza las etiquetas (one-hot encoding)
y_true_bin = label_binarize(y_test, classes=[0, 1, 2, 3])

# Inicializa las variables para almacenar las tasas TPR y FPR para cada clase
fpr = dict()
tpr = dict()
roc_auc = dict()

# Calcula las tasas TPR y FPR para cada clase
for i in range(4):  # 4 clases en tu caso
    fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_prob[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Tamaño de la figura
plt.figure(figsize=(8, 6))

# Plotea las curvas ROC para cada clase
for i in range(4):
    plt.plot(fpr[i], tpr[i], label=f'Clase {i} (AUC = {roc_auc[i]:.2f}')

# Configuración de la figura
plt.plot([0, 1], [0, 1], 'k--', label='Aleatorio')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Tasa de Falsos Positivos (FPR)')
plt.ylabel('Tasa de Verdaderos Positivos (TPR)')
plt.title('Curva ROC Multiclase')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# Instantiation of the explainer - GradCam()
explainer = GradCAM()  # TODO: Asegúrate de que GradCAM esté correctamente definido en tu código

# Obtener la etiqueta (clase) de la imagen
index = 5   # Cambiar '1' por cualquier número entre 0 y len(y_test) - 1
label = int(y_test[index])

# Expandir las dimensiones de la imagen para que coincida con las expectativas de la red
img_to_explain = np.expand_dims(x_test[index], 0)

# Llamada al método explain()
output = explainer.explain(validation_data=[img_to_explain, None],  # Cambié 'validation_data' por [img_to_explain, None]
                           model=modelo, 
                           class_index=label)

# Visualizar la imagen
fig,ax = plt.subplots(1,2)
ax[0].imshow(x_test[index].squeeze())
ax[1].imshow(output)
ax[0].set_title(label)
ax[1].set_title(label)
plt.show()

In [ ]:
# Instantiation of the explainer - GradCam()
explainer = GradCAM()  # Asegúrate de que GradCAM esté correctamente definido en tu código

# Obtener 10 índices aleatorios para seleccionar 10 imágenes
sample_indices = np.random.choice(len(y_test), size=10, replace=False)

# Iterar sobre los índices de la muestra
for index in sample_indices:
    # Obtener la etiqueta (clase) de la imagen
    label = int(y_test[index])

    # Expandir las dimensiones de la imagen para que coincida con las expectativas de la red
    img_to_explain = np.expand_dims(x_test[index], 0)

    # Llamada al método explain()
    output = explainer.explain(validation_data=[img_to_explain, None],  # Cambié 'validation_data' por [img_to_explain, None]
                               model=modelo, 
                               class_index=label)

    # Visualizar la imagen y su explicación GradCAM
    fig, ax = plt.subplots(1, 2)
    ax[0].imshow(x_test[index].squeeze(), cmap='gray')  # Asumo que las imágenes son en escala de grises
    ax[1].imshow(output, cmap='jet')  # Puedes ajustar el mapa de colores según tus preferencias
    ax[0].set_title(f'Clase: {label}')
    ax[1].set_title(f'GradCAM para Clase: {label}')
    plt.show()

In [ ]:
# Crear modelo ResNet50
model_resnet = Sequential()
model_resnet.add(ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3)))
model_resnet.add(MaxPooling2D())
model_resnet.add(Flatten())
model_resnet.add(Dropout(0.3))
model_resnet.add(Dense(150, activation='relu'))
model_resnet.add(Dense(4, activation='softmax'))
model_resnet.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])


# Guardar el modelo como imagen
plot_model(model_resnet, to_file='/kaggle/working/resnet.png')

In [ ]:
model_resnet.summary()